In [25]:
import pandas as pd
import time
import numpy as np
import lyricsgenius
from fuzzywuzzy import fuzz

In [42]:
top_lyrics = pd.read_csv('WDSS_Kpop_Translations.csv')
top_lyrics['translated_lyrics'] = ['' for x in top_lyrics['translated_lyrics']]

In [44]:
##identify all songs missing lyrics
top_lyrics['missing_lyrics'] = top_lyrics['original_lyrics'].isna()

##it looks like songs with parentheses were missed at a high rate, so we'll try removing those and running again
top_lyrics["song_title"] = top_lyrics["song_title"].str.replace(r"\(.*\)", "")
#this helped with a few songs, but many remain without lyrics
#may fill in manually but will leave for now


top_lyrics.loc[top_lyrics['missing_lyrics'] == True]

,Unnamed: 0,Unnamed: 0.1,chart_date,artist,song_title,original_lyrics,translated_lyrics,peak_position,entry_weeks,Total\nweeks,main_artist,missing_lyrics
7,7,25,2011-07-09,2:00 PM,Hands Up,NaN,,3,5,5,2:00 PM,True
21,21,93,2012-06-30,f(x),Electric Shock,NaN,,3,13,13,f(x),True
22,22,97,2012-07-21,2NE1,I Love You,NaN,,3,10,10,2NE1,True
24,24,112,2012-09-22,Ultimate Pop Hits!,Gangnam Style,NaN,,2,4,4,Ultimate Pop Hits!,True
60,60,432,2015-08-22,G-Dragon and T.O.P,Zutter,NaN,,2,13,13,G-Dragon,True
72,72,496,2016-01-30,Bae Suzy & Baekhyun,Dream,NaN,,3,2,2,Bae Suzy & Baekhyun,True
79,79,529,2016-04-09,K.Will,Taeyangyi Huye,NaN,,3,6,6,K.Will,True
96,96,615,2016-09-17,NCT Dream,Chewing Gum,NaN,,2,2,2,NCT Dream,True
99,99,623,2016-10-01,MOBB feat. KUSH,Hit Me,NaN,,3,1,1,MOBB,True
111,111,669,2016-12-24,Park Chanyeol & Punch,Stay With Me,NaN,,3,3,10,Park Chanyeol & Punch,True


In [19]:
##search for original lyrics on genius
genius = lyricsgenius.Genius('')

In [20]:
for artist_name in top_lyrics['main_artist'].unique():
    print(artist_name)
    #locate all songs the artist has in the top songs
    artist_songs = top_lyrics.loc[top_lyrics['main_artist'] == artist_name]
    #count number of lyrics for this artist that haven't been filled in
    missing_lyrics = len(artist_songs.loc[artist_songs['missing_lyrics'] == True])
    #rerun genius if lyrics are missing
    if missing_lyrics > 1:
        try: #search for artist on genius, skip if not found
            #seearch for top songs for artist on genius, sorted by popularity -- should mostly match song chart positions
            artist_genius = genius.search_artist(artist_name, max_songs=len(artist_songs) + 5, sort="popularity")
            ##search for each song on genius and store lyrics
            for i,r in artist_songs.iterrows():
                print(r['song_title'])
                song_genius = artist_genius.song(r['song_title'])
                #store song lyrics if possible, otherwise skip
                try:
                    lyrics_raw = song_genius.lyrics
                    print(lyrics_raw[:30])
                    top_lyrics.at[i, 'original_lyrics'] = lyrics_raw
                except:
                    pass
        except:
            pass

Shinee
2NE1
2PM
Jaejoong
Big Bang
Jay Park
2:00 PM
miss A
Super Junior
Wonder Girls
IU
Block B
f(x)
Psy
Ultimate Pop Hits!
Girls' Generation
B.A.P
BTS
Searching for songs by BTS...

Changing artist name to 'BTS (방탄소년단)'
Song 1: "Dynamite"
Song 2: "FAKE LOVE"
Song 3: "MIC Drop (Steve Aoki Remix)"
Song 4: "전하지 못한 진심 (The Truth Untold)"
Song 5: "봄날 (Spring Day)"
Song 6: "DNA"
Song 7: "MIC Drop (Steve Aoki Remix) [Desiigner Remix]"
Song 8: "Intro: Serendipity (세렌디피티)"
Song 9: "Euphoria"
Song 10: "Airplane pt.2"
Song 11: "Magic Shop"
Song 12: "피 땀 눈물 (Blood Sweat & Tears)"
Song 13: "Crystal Snow"
Song 14: "MIC Drop"
Song 15: "Anpanman"
Song 16: "Intro: Singularity"
Song 17: "작은 것들을 위한 시 (Boy With Luv)"
Song 18: "BEGIN"
Song 19: "ON"
Song 20: "고민보다 Go (Go Go)"
Song 21: "땡 (Ddaeng)"
Song 22: "Best of Me"
Song 23: "낙원 (Paradise)"
Song 24: "Outro: Tear"
Song 25: "Not Today"
Song 26: "Love Maze"
Song 27: "LIE"
Song 28: "Don’t Leave Me"
Song 29: "Let Go"
Song 30: "134340"
Song 31: "Life Goes On"


In [45]:
#### access artist object for all genius songs translated into english
translations = genius.search_artist('Genius English Translations', sort='popularity', max_songs=1)

Searching for songs by Genius English Translations...

Song 1: "Luis Fonsi & Daddy Yankee - Despacito (Remix) ft. Justin Bieber (English Translation)"

Reached user-specified song limit (1).
Done. Found 1 songs.


In [29]:
##test searching for new song and accessing lyrics
test_song = translations.song("Paldogangsan")
print(test_song)
print(test_song.lyrics)
#confirm partial ratio is high to avoid mismatches
print(fuzz.partial_ratio("Paldogangsan", test_song.title))
#it successfully identified one new song so we will use this to try to find english translations for the other songs in the spreadsheet

Searching for "Paldogangsan" by Genius English Translations...
Done.
"팔도강산 Paldogangsan (Satoori Rap) (English Translation)" by Genius English Translations:
    [Intro: RM]
    Yo once again
    Big Hit Represent
    We are BTS Let’s Go
    
    [Chorus: RM, J-Hope, Suga]
    From Seo...
[Intro: RM]
Yo once again
Big Hit Represent
We are BTS Let’s Go

[Chorus: RM, J-Hope, Suga]
From Seoul to Gangwon, Kyeongsangdo
Chungcheong to Jeolla
What is he saying (what)
What is he saying (what)
From Seoul to Gangwon, Kyeongsangdo
Chungcheong to Jeolla
Tell them that we have come (what)
We’re real hot stuff

[Verse 1: Suga]
Hello misters
My hometown is Daegu
So today, I’ll rap in satoori
Let’s play
Not caring about

[J-Hope]
The other boys and girls
Hello everyone
Oh my, what is this?
Why are you
Rapping like that?
Misters and Mrs.’s
Don’t just stand there
Come out and have fun

[Verse 2: Suga]
When it comes to Kyeongsangdo
The successors of Silla’s Hwarang
Keep growing
When it comes to satoori
Ky

In [49]:
for i, r in top_lyrics.iterrows():
    title = r['song_title']
    artist = r['main_artist']
    #only run for songs that don't have translated lyrics stored (for re-runs)
    if r['translated_lyrics'] == '':
        print(title)
        song_trans = translations.song(artist + ' - ' + title)
        #only run if results turn something up and song object isn't empty
        if song_trans:
            similarity = fuzz.partial_ratio(artist + ' - ' + title, song_trans.title)
            print(artist, title, " | ", song_trans.title, " | ", similarity)
            #if similarity is above threshold, store translated lyrics
            if similarity > 60:
                top_lyrics.at[i, 'translated_lyrics'] = song_trans.lyrics

(How Can I Say)(English Translation)  |  75
Never Ever
Searching for "Got7 - Never Ever" by Genius English Translations...
Done.
Got7 Never Ever  |  The Marx-Engels Reader (Chapter 4)  |  41
Who's It Gonna Be
Searching for "BgA (*K-pop parody group) - Who's It Gonna Be" by Genius English Translations...
No results found for: 'BgA (*K-pop parody group) - Who's It Gonna Be Genius English Translations'
Really Really
Searching for "Winner - Really Really" by Genius English Translations...
Done.
Winner Really Really  |  WINNER - 릴리릴리 (Really Really) (English Translation)  |  73
Rumor
Searching for "K.A.R.D - Rumor" by Genius English Translations...
No results found for: 'K.A.R.D - Rumor Genius English Translations'
Signal
Searching for "Twice - Signal" by Genius English Translations...
Done.
Twice Signal  |  TWICE - SIGNAL (English Translation)  |  36
Don't Wanna Cry
Searching for "Seventeen - Don't Wanna Cry" by Genius English Translations...
Done.
Seventeen Don't Wanna Cry  |  SEVENTEEN -

In [58]:
has_trans_lyrics = top_lyrics.loc[top_lyrics['translated_lyrics'] != '']
print('found', len(has_trans_lyrics), 'lyric translations')

has_both_lyrics = has_trans_lyrics.loc[has_trans_lyrics['missing_lyrics'] == False]
print(len(has_both_lyrics), 'have both translated and original lyrics')

print(len(top_lyrics) - len(has_both_lyrics), 'songs are missing one set of lyrics')

found 127 lyric translations
117 have both translated and original lyrics
128 songs are missing one set of lyrics


In [39]:
top_lyrics.to_csv('WDSS_Kpop_Translations.csv')